In [1]:
import pandas as pd
from scipy.io import loadmat
import numpy as np
import scipy.stats as s
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [2]:
Data = pd.read_csv('mnist_train.csv')


In [3]:
sc = MinMaxScaler()

TrainData = Data
X_Train_Data = TrainData.iloc[:,1:]
Y_Train_Data = TrainData.iloc[:,0]
X_Train_Data = sc.fit_transform(X_Train_Data)
print(X_Train_Data)

TestData = pd.read_csv('mnist_test.csv')
X_Test_Data = TestData.iloc[:,1:]
Y_Test_Data = TestData.iloc[:,0]
X_Test_Data = sc.fit_transform(X_Test_Data)
print(X_Test_Data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
def PCA(Train_Data,Test_Data):
    Cov = np.cov(Train_Data.T)
    eigen_pairs = {}
    eigen_vals,eigen_vecs = np.linalg.eig(Cov)

    for i in range(len(eigen_vals)):
        eigen_pairs[eigen_vals[i]] = eigen_vecs[i] 

    tot = sum(eigen_vals)

    var_exp = [(i/tot) for i in sorted(eigen_vals,reverse= True)]
    cum_var_exp = np.cumsum(var_exp)


    for i in range(len(cum_var_exp)):
        if cum_var_exp[i] >= 0.98:
            k = i
            break;

    print("K :",k)
    print("LEN: ",len(eigen_vals))


    sorted(eigen_vals,reverse=True)


    w = []
    for i in eigen_vals:
        w.append(np.real(eigen_pairs[i][:k]))

    w = np.array(w)


    print("w.shape",w.shape)
    X_Train_Data_PCA = Train_Data.dot(w)
    print("X_Train_Data_PCA.shape",X_Train_Data_PCA.shape)

    X_Test_Data_PCA = Test_Data.dot(w)
    print("X_Test_Data_PCA.shape",X_Test_Data_PCA.shape)
    
    return X_Train_Data_PCA,X_Test_Data_PCA

In [5]:
PCapList =[Y_Train_Data[Y_Train_Data == i].shape[0]/60000 for i in range(10)]
PCapList


[0.09871666666666666,
 0.11236666666666667,
 0.0993,
 0.10218333333333333,
 0.09736666666666667,
 0.09035,
 0.09863333333333334,
 0.10441666666666667,
 0.09751666666666667,
 0.09915]

In [6]:
NumberList = {}
for i in range(10):
    NumberList[i] = X_Train_Data[Y_Train_Data == i]
    print(NumberList[i].shape)

(5923, 784)
(6742, 784)
(5958, 784)
(6131, 784)
(5842, 784)
(5421, 784)
(5918, 784)
(6265, 784)
(5851, 784)
(5949, 784)


In [7]:
def calProb(Test_Data_PCA,Pcap,Mean,cov):
    return Pcap * s.multivariate_normal.pdf(Test_Data_PCA,Mean,cov)

ProbList = []
for i in range(10):
    X_Train_Data_PCA,X_Test_Data_PCA = PCA(NumberList[i],X_Test_Data)
    Mean = np.mean(a = X_Train_Data_PCA,axis=0)
    print(Mean.shape)
    Cov = np.cov(X_Train_Data_PCA.T)
    print(Cov.shape)
    ProbList.append(calProb(X_Test_Data_PCA,PCapList[i],Mean,Cov))

    
ProbList   



K : 204
LEN:  784
w.shape (784, 204)
X_Train_Data_PCA.shape (5923, 204)
X_Test_Data_PCA.shape (10000, 204)
(204,)
(204, 204)
K : 129
LEN:  784
w.shape (784, 129)
X_Train_Data_PCA.shape (6742, 129)
X_Test_Data_PCA.shape (10000, 129)
(129,)
(129, 129)
K : 236
LEN:  784
w.shape (784, 236)
X_Train_Data_PCA.shape (5958, 236)
X_Test_Data_PCA.shape (10000, 236)
(236,)
(236, 236)
K : 226
LEN:  784
w.shape (784, 226)
X_Train_Data_PCA.shape (6131, 226)
X_Test_Data_PCA.shape (10000, 226)
(226,)
(226, 226)
K : 223
LEN:  784
w.shape (784, 223)
X_Train_Data_PCA.shape (5842, 223)
X_Test_Data_PCA.shape (10000, 223)
(223,)
(223, 223)
K : 217
LEN:  784
w.shape (784, 217)
X_Train_Data_PCA.shape (5421, 217)
X_Test_Data_PCA.shape (10000, 217)
(217,)
(217, 217)
K : 196
LEN:  784
w.shape (784, 196)
X_Train_Data_PCA.shape (5918, 196)
X_Test_Data_PCA.shape (10000, 196)
(196,)
(196, 196)
K : 206
LEN:  784
w.shape (784, 206)
X_Train_Data_PCA.shape (6265, 206)
X_Test_Data_PCA.shape (10000, 206)
(206,)
(206, 206)


[array([2.80696399e+30, 1.08994144e+07, 3.54677177e+20, ...,
        1.35850590e-18, 1.78885344e-59, 1.68107694e-71]),
 array([1.37906682e-237, 2.62448737e-285, 8.68112553e+072, ...,
        6.52818104e-228, 1.18321377e-249, 0.00000000e+000]),
 array([4.32296581e+43, 1.90051960e+38, 3.66410830e+48, ...,
        4.61656351e+18, 3.58531034e-48, 7.76921477e-22]),
 array([4.39757096e+61, 3.69505960e-03, 3.19544672e+40, ...,
        4.39355878e+39, 9.33297022e-02, 8.27745206e-54]),
 array([3.03809448e-004, 1.76453636e-210, 4.63320259e+064, ...,
        1.11794631e+068, 2.50735807e-089, 9.63682088e-138]),
 array([5.64483161e-05, 2.45079285e-59, 1.34306976e+27, ...,
        3.83135942e+08, 2.91803022e+49, 5.14941833e-55]),
 array([6.85188502e-92, 1.71042659e-06, 7.76311749e+39, ...,
        4.57863009e-08, 2.41491485e-43, 6.09756687e+19]),
 array([5.83745952e+106, 2.13383596e-221, 3.11478375e+076, ...,
        3.58783918e+042, 7.49371027e-111, 6.23812895e-183]),
 array([2.39922367e+050, 4.455

In [8]:
  
    
Predicted = []
All_Tot = sum(ProbList)

Pred_Prob = []
for i in range(10):
    Pred_Prob.append(ProbList[i]/All_Tot)

Pred_Prob = np.array(Pred_Prob)
# print(Pred_Prob)

for j in range(len(X_Test_Data_PCA)):
    tslist = list(Pred_Prob[:,j])
#     print(sum(tslist))
#     print(tslist)
    idx = tslist.index(max(tslist))
#     print(idx," : ", Y_Test_Data[j] )
    Predicted.append(idx)
    
count = 0
for i in range(len(Y_Test_Data)):
    if Predicted[i] == Y_Test_Data[i]:
        count += 1
print("Accuracy: ",(count/len(Y_Test_Data))*100)

Accuracy:  75.86


### Using SkLearn

#### Without Dimensionality 

In [9]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(X_Train_Data,Y_Train_Data)

predict_by_model = classifier.predict(X_Test_Data)


In [10]:
# for i in range(len(predict_by_model)):
#     print(predict_by_model[i],Y_Test_Data[i])

accuracy = sum(predict_by_model == Y_Test_Data)/(Y_Test_Data.shape[0])
print('Accuracy is: {}%'.format(accuracy*100))

Accuracy is: 55.46%


#### With Dimensionality Reduction 

In [11]:
X_Train_Data_PCA,X_Test_Data_PCA = PCA(X_Train_Data,X_Test_Data)

K : 260
LEN:  784
w.shape (784, 260)
X_Train_Data_PCA.shape (60000, 260)
X_Test_Data_PCA.shape (10000, 260)


In [12]:

classifier.fit(X_Train_Data_PCA,Y_Train_Data)

predict_by_model_PCA = classifier.predict(X_Test_Data_PCA)

In [13]:
accuracy = sum(predict_by_model_PCA == Y_Test_Data)/(Y_Test_Data.shape[0])
print('Accuracy is: {}%'.format(accuracy*100))

Accuracy is: 83.63000000000001%
